In [10]:
import pandas as pd
from pandas import json_normalize
# If you have a JSON string, you can convert it into a dictionary first
import json
with open('data.json', 'r') as f:
    data = json.load(f)

# Use json_normalize to convert nested JSON to a flat table
flat_data = json_normalize(data, record_path=['data'])
flat_data



,content,id,ref,type,bab,bagian,paragraf,pasal,level,context,additional_context,chunks,source_token_length,buku,alias,term
0,,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,"BERITA NEGARA REPUBLIK INDONESIA No.920, 2017 ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,PERATURAN MENTERI KEUANGAN REPUBLIK INDONESIA ...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,DENGAN RAHMAT TUHAN YANG MAHA ESA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,"MENTERI KEUANGAN REPUBLIK INDONESIA,",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2342,SRI MULYANI INDRAWATI,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2343,Diundangkan di Jakarta pada tanggal 7 Juli 2017,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2344,DIREKTUR JENDERAL PERATURAN PERUNDANG-UNDANGAN...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2345,ttd,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
flat_data

,content,id,ref,type,bab,bagian,pasal,level
5,Menimbang :,heading_menimbang,heading_menimbang,konsideran,None,None,None,1.0
7,Mengingat :,heading_mengingat,heading_mengingat,konsideran,None,None,None,1.0
9,MEMUTUSKAN:,heading_memutuskan,heading_memutuskan,konsideran,None,None,None,1.0
11,"KEDUDUKAN, TUGAS, DAN FUNGSI",bab-i,BAB I,heading_bab,None,None,None,1.0
12,,pasal-1,Pasal 1,heading_pasal,bab-i,None,pasal-1,4.0
...,...,...,...,...,...,...,...,...
2329,,pasal-37,Pasal 37,heading_pasal,bab-xi,None,pasal-37,4.0
2331,KETENTUAN PENUTUP,bab-xi,BAB XI,heading_bab,None,None,None,1.0
2332,,pasal-38,Pasal 38,heading_pasal,bab-xi,None,pasal-38,4.0
2334,,pasal-39,Pasal 39,heading_pasal,bab-xi,None,pasal-39,4.0


In [11]:
pip install transformers pandas


Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.1 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [12]:
#print all column name
print(flat_data.columns)

Index(['content', 'id', 'ref', 'type', 'bab', 'bagian', 'paragraf', 'pasal',
       'level', 'context', 'additional_context', 'chunks',
       'source_token_length', 'buku', 'alias', 'term'],
      dtype='object')


In [15]:
import pandas as pd
import json
import re
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline

# Set up the IndoBERT NER model
model_name = "indobenchmark/indobert-base-p1"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForTokenClassification.from_pretrained(model_name)
ner_pipeline = pipeline("ner", model=model, tokenizer=tokenizer, aggregation_strategy="simple")

Some weights of BertForTokenClassification were not initialized from the model checkpoint at indobenchmark/indobert-base-p1 and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [24]:
import pandas as pd
import json
import re
import torch
from transformers import AutoTokenizer, AutoModelForTokenClassification
from transformers import pipeline


# Function to identify monetary amounts using regex
def extract_money(text):
    money_pattern = re.compile(r'Rp\s?\d{1,3}(?:\.\d{3})*(?:,\d{0,2})?\s?')
    return money_pattern.findall(text)

# Function to identify key dates using regex
def extract_dates(text):
    date_pattern = re.compile(r'\b(?:\d{1,2}\s(?:Januari|Februari|Maret|April|Mei|Juni|Juli|Agustus|September|Oktober|November|Desember)\s\d{4})\b')
    return date_pattern.findall(text)

# Function to identify prohibitions using regex
def extract_prohibitions(text):
    prohibition_pattern = re.compile(r'\b(dilarang)\b:?\s*(\w\.\s*[^;]+;?|\w\.\s*[^;]+dan)', re.IGNORECASE | re.MULTILINE)
    prohibitions = prohibition_pattern.findall(text)
    prohibitions_list = [item[1].strip() for item in prohibitions]
    return prohibitions_list

# Function to extract key terms for summarization
def extract_key_terms(entity_list):
    return [entity['word'] for entity in entity_list if entity['entity_group'] in ('TERM', 'KEYWORD')]

# Function to apply NER to given text with truncation to the model's max length
# Updated apply_ner function to work safely with transformers pipeline
def apply_ner(text, tokenizer, model):
    # Perform NER prediction only on non-empty text
    ner_results = []
    if text.strip():  # Ensure text is not empty
        tokenized_input = tokenizer(text, truncation=True, max_length=512, padding='max_length', return_tensors="pt")
        output = model(**tokenized_input)
        ner_result = [model.config.id2label[prediction] for prediction in output.logits.argmax(dim=-1).flatten().tolist()]
        ner_results = ner_pipeline(tokenized_input)
    return ner_results

flat_data = flat_data.dropna(subset=['content'])

# Updated process_record function
def process_record(record, tokenizer, model):
    content = record['content']

    # Check if content is empty and handle it appropriately
    if not content:
        return {
            'content': content,
            'money': [],
            'dates': [],
            'key_terms': [],
            'prohibitions': []
        }

    record['money'] = extract_money(content)
    record['dates'] = extract_dates(content)
    record['prohibitions'] = extract_prohibitions(content)
    ner_results = apply_ner(content, tokenizer, model)
    record['key_terms'] = extract_key_terms(ner_results)

    return record

# Apply the process_record function to each row in the DataFrame
processed_data = flat_data.apply(lambda x: process_record(x, tokenizer, model), axis=1)

# Display the results or continue with more 
print(processed_data[['content', 'money', 'dates', 'key_terms', 'prohibitions']])


ValueError: At least one input is required.